# Introduktion til `spaCy` 

"spaCy" indeholder forskellige sprogmodeller - herunder en dansk sprogmodel.

Overordnet virker spaCy ved, at man specificerer en sprogmodel samt nogen "processors", som modellen skal indeholde. 

SpaCy's sprogmodeller indeholder blandt andet:
- Tokenizer (inddeling i enkeltord)
- Lemmatizer (konvertering til navneform)
- Part-Of-Speech tagging (POS-tagging) (identificering af ordtyper)
- Dependency parsing (sætningskonstruktion)
- Named-Entity-Recognition (NER) (udledning af "named entities", fx personer og organisationer)

## Brug af spaCy i Python

1. Indlæs sprogmodel
2. Analysér tekstykke
3. Inspicér resultater

In [1]:
import spacy

#!python -m spacy download 'da_core_news_sm' # evt. installer sprogmodel

Når sprogmodellen er hentet, kan vi bruge den ved at indlæse modellen. Som standard indlæses modellen med alle processerne, men det er muligt at aktivere/deaktivere specifikke processer.

Efter modellen er defineret, kan man lade sprogmodellen analysere tekst.

In [2]:
nlp = spacy.load("da_core_news_sm") # Definerer model

doc = nlp('Politiet har givet borgerne råd') # Analyserer tekst med model

Når modellen anvendes på et stykke tekst, behandler den tekststykket med de forskellige processors, som er en del af sprogmodellen (som standard for dansk: tokenizer, part-of-speech tagging, lemmatizer og dependency parsing).

Outputtet (`doc`) indeholder de forskellige værdier, som er udledt af teksten, som attributes (et attribute for token, et for lemma, et for POS-tag osv.).

Vi kan fx visualisere sætningskonstruktionen med funktionen `displacy`:

In [5]:
from spacy import displacy # skal indlæses separat
from IPython.display import display, HTML

display(HTML(displacy.render(doc, style='dep')))

## Lemmatizing

Et ords "lemma" er dets grammatiske stamme (fx "er"->"være", "spiste"->"spise"). SpaCy's sprogmodeller indeholder typisk en indbygget ordbog til at finde stammen for de enkelte ord. Et ords "lemma" er gem under attributtet `.lemma_` for hvert ord:

In [6]:
for word in doc:
    print(f'{word.text:<15} {word.lemma_}')

Politiet        politi
har             have
givet           give
borgerne        borger
råd             råd


## Part-of-speech tags

SpaCy tagger automatisk hvert ord med sin ordklasse ("part-of-speech"-tag/POS-tag). Disse er gemt under attributtet `.pos_` for hvert ord:

In [7]:
for word in doc:
    print(f'{word.text:<15} {word.pos_}')

Politiet        NOUN
har             AUX
givet           VERB
borgerne        NOUN
råd             NOUN


Part-of-speech tagging virker ved, at modellen i forvejen er trænet på danske tekster, og derfor har "set" de forskellige ord i kontekst før. Som det kan ses, er modellen dog ikke perfekt (fx "trygge" er angivet som navneord (NOUN), selvom der her er tale om et tillægsord (ADJ)).

Part-of-speech tagging tillader fx at isolere visse ord i et stykke tekst:

In [10]:
keep_tags = ['VERB']
keep_words = []

for word in doc:
    if word.pos_ in keep_tags:
        keep_words.append(word)

for word in keep_words:
    print(f'{word.text:<15} {word.pos_}')

givet           VERB


## Dependency parsing

SpaCy laver også analyse af sætningskonstruktion (dependency parsing). Hvilken del af sætningen, som ordene er analyseret frem til, kan tilgås af attribut `.dep_`.

In [11]:
for word in doc:
    print(f'{word.text:<15} {word.dep_}')

Politiet        nsubj
har             aux
givet           ROOT
borgerne        iobj
råd             ROOT


## Named entities

"Named entities" kan groft sagt forstås som "meningsfulde enheder" i teksten. Det kan fx være personer, organisationer eller steder. Ligesom ved part-of-speech tagging, fungerer "named entity recognition" ved, at modellen enten har set disse enheder før eller er bekendt med, hvordan sådanne enheder fremgår i sætningen (hvor er de i sætningskonstruktionen, hvilke ordklasser er de associeret med).

Alle ord i en tekst er ikke en "named entity". Named entities kan tilgås gennem attributtet `ents` for det behandlede stykke tekst (`doc`). Fra dette kan ses, hvilke enheder er udledt, og hvordan de er kategoriseret:

In [12]:
doc = nlp("Søs Marie Serup, politisk kommentator og tidligere særlig rådgiver for Løkke, fortalte i fredags i DR's nyhedspodcast 'Genstart' om hans evne til altid at komme tilbage i politik.")

for ent in doc.ents:
    print(f'{ent.text:<15} {ent.label_}')

Marie Serup     PER
Løkke           LOC
DR's            ORG


Denne sprogmodel arbejder med fire named entity tags:
- LOC: Steder
- ORG: Organisationer
- PER: Personer
- MISC: Andet

Af ovenstående ses, at modellen identificerer "DR" som en organisation, hvilket er meget passende. Derudover genkender den "Marie Serup" som person, men har udeladt fornavnet "Søs". Dog er "Løkke" fejlklassificeret som et sted (LOC).

## Tilpas spaCy pipeline

Når man definerer `nlp`-funktionen (sit spaCy pipeline) med en sprogmodel (`spacy.load()`), inkluderes alle komponenter som standard. Hvis man kun er interesseret i specifikke komponenter, kan man slå dele af pipeline til eller fra. 

Se de forskellige komponenter her: [https://spacy.io/usage/processing-pipelines#built-in](https://spacy.io/usage/processing-pipelines#built-in).

Når man arbejder med store mængder tekstdata, kan det give mening at forsimple funktionen for at spare beregningstid.

### Slå komponenter fra

Man slår komponenter fra ved brug af argumentet `disable`, når man indlæser modellen.

In [13]:
nlp = spacy.load("da_core_news_sm", disable = ['parser'])

doc = nlp('Politiet har givet borgerne råd') # analysér tekststykke med nyt pipeline

`doc` indeholder stadig lemma, da lemmatizer stadig er slået til.

In [14]:
for word in doc:
    print(f'{word.text:<15} {word.lemma_}')

Politiet        politi
har             have
givet           give
borgerne        borger
råd             råd


Der er ikke længere nogen dependency labels, da `parser` er slået fra (returnerer None).

In [15]:
for word in doc:
    print(f'{word.text:<15} {word.dep_}')

Politiet        
har             
givet           
borgerne        
råd             


### Slå komponenter til

Man slår komponenter til ved brug af argumentet `enable`, når man indlæser modellen. Alle komponenter, som ikke listes, slås fra.

In [16]:
nlp = spacy.load("da_core_news_sm", enable = ['parser'])

doc = nlp('Politiet har givet borgerne råd') # analysér tekststykke med nyt pipeline

`doc` indeholder nu ikke lemma, da kun parser er slået til.

In [17]:
for word in doc:
    print(f'{word.text:<15} {word.lemma_}')

Politiet        
har             
givet           
borgerne        
råd             


Der er dependency labels, da parser er slået til.

In [18]:
for word in doc:
    print(f'{word.text:<15} {word.dep_}')

Politiet        case
har             case
givet           case
borgerne        ROOT
råd             punct


### Tokenizer som særskilt funktion

Hvis man blot vil bruge tokenizeren, kan denne tilgås direkte.

In [19]:
nlp = spacy.load("da_core_news_sm")

tokenizer = nlp.tokenizer

In [20]:
tokenizer('Politiet har givet borgerne råd')

Politiet har givet borgerne råd

Tokenizeren returnerer stadig et `doc` objekt, men da den kun giver tokens tilbage, kan man tvinge output om til en liste:

In [21]:
list(tokenizer('Politiet har givet borgerne råd'))

[Politiet, har, givet, borgerne, råd]

## spaCy pipeline på flere stykker tekst

Pipeline-funktionen (`nlp`) virker kun på ét stykke tekst. Afhængig af datastruktur, kan man anvende pipeline på flere stykker tekst.

In [22]:
texts = [
    'Smileyordningen får stor makeover: Kontrolrapporten forsvinder og en QR-kode kommer til',
    'Bro skal rives ned: Motorvej spærres i 14 timer',
    'Indonesien er klar med Sydøstasiens første højhastighedstog',
    'Politiet dropper efterforskning af hospital og region efter kræftskandale',
    'Hvad foregår der i Ikast? De kom med på et wildcard, og nu topper de hele baduljen'
]

### Lister

Hvis tekster er i en liste, kan man bruge metoden `.pipe()` til at anvende pipeline på flere tekststykker.

`.pipe()` returnerer et "generator object". Dette er en speciel type objekt i Python, som kun giver et output, når den bliver kaldt (en måde at spare hukommelse). Hvis vi fx vil have teksterne tilbage som en liste af `doc`-objekter, kan man tvinge output om til en liste:

In [23]:
docs = list(nlp.pipe(texts))

In [24]:
for word in docs[0]:
    print(f'{word.text:<20} {word.pos_}')

Smileyordningen      NOUN
får                  VERB
stor                 ADJ
makeover             NOUN
:                    PUNCT
Kontrolrapporten     NOUN
forsvinder           VERB
og                   CCONJ
en                   DET
QR-kode              NOUN
kommer               VERB
til                  ADP


In [25]:
for word in docs[1]:
    print(f'{word.text:<20} {word.pos_}')

Bro                  ADV
skal                 AUX
rives                VERB
ned                  ADV
:                    PUNCT
Motorvej             PROPN
spærres              VERB
i                    ADP
14                   NUM
timer                NOUN


### Pandas Series

Hvis tekster er i en pandas Series, kan man bruge metoden `.apply()` i pandas. Dog forventer `.apply()`, at der kun gives ét output. Derfor bør man lave en wrapper-funktion, så man kan udpege, hvad der skal gives som output:

In [26]:
# dan funktion til at hente named entities

def get_ents(text):
    doc = nlp(text)

    ents = list(doc.ents)

    return(ents)

In [27]:
import pandas as pd

texts_s = pd.Series(texts) # konvertér liste til series

texts_s.apply(get_ents) # anvend funktion på series

0       [(QR-kode)]
1      [(Motorvej)]
2    [(Indonesien)]
3                []
4         [(Ikast)]
dtype: object

# Arbejde med større pandas dataframes

In [ ]:
df = pd.read_csv('/Users/jeppefl/Library/CloudStorage/OneDrive-AalborgUniversitet/01_work/01_undervisning/02_sds1/03_data/spam.csv', encoding='cp1252')

df = df[['v1', 'v2']].copy()
df.columns = ['label', 'message']
df = df.dropna()

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
# Indlæs engelsk model (da beskederne er på engelsk)
nlp = spacy.load('en_core_web_sm')
print(f"Pipeline komponenter: {nlp.pipe_names}")


## Tilgang 1

In [ ]:
# Tag et lille subset først for at teste
df_small = df.head(100).copy()

In [ ]:
# Simpel wrapper funktion
def process_with_spacy(text):
    return nlp(text)

df_small['doc'] = df_small['message'].apply(process_with_spacy)

In [ ]:
first_doc = df_small['doc'].iloc[0]
print(f"Tekst: {first_doc.text[:100]}...")
print(f"Antal tokens: {len(first_doc)}")
print(f"Antal sætninger: {len(list(first_doc.sents))}")

## Tilgang 2

In [ ]:
df_medium = df.head(500).copy()

In [ ]:
docs = list(nlp.pipe(df_medium['message']))
df_medium['doc'] = docs

## TOKENS OG LEMMAS

In [ ]:
print("\nTokens og lemmas:")

def extract_tokens(doc):
    return [token.text for token in doc]

def extract_lemmas(doc):
    return [token.lemma_ for token in doc]

df_medium['tokens'] = df_medium['doc'].apply(extract_tokens)
df_medium['lemmas'] = df_medium['doc'].apply(extract_lemmas)
df_medium['n_tokens'] = df_medium['tokens'].apply(len)

print(df_medium[['message', 'n_tokens']].head())

## PART-OF-SPEECH TAGS


In [ ]:
print("\nPart-of-speech tags:")

def extract_pos_tags(doc):
    return [token.pos_ for token in doc]

def count_verbs(doc):
    return sum(1 for token in doc if token.pos_ == 'VERB')

def count_nouns(doc):
    return sum(1 for token in doc if token.pos_ == 'NOUN')

df_medium['pos_tags'] = df_medium['doc'].apply(extract_pos_tags)
df_medium['n_verbs'] = df_medium['doc'].apply(count_verbs)
df_medium['n_nouns'] = df_medium['doc'].apply(count_nouns)

print(df_medium[['message', 'n_verbs', 'n_nouns']].head())

## NAMED ENTITIES


In [ ]:
print("\nNamed entities:")

def extract_entities(doc):
    return [(ent.text, ent.label_) for ent in doc.ents]

def count_entities(doc):
    return len(doc.ents)

df_medium['entities'] = df_medium['doc'].apply(extract_entities)
df_medium['n_entities'] = df_medium['doc'].apply(count_entities)

print(df_medium[['message', 'entities']].head(10))

## SÆTNINGER

In [ ]:
print("\nSætninger:")

def extract_sentences(doc):
    return [sent.text for sent in doc.sents]

def count_sentences(doc):
    return len(list(doc.sents))

df_medium['sentences'] = df_medium['doc'].apply(extract_sentences)
df_medium['n_sentences'] = df_medium['doc'].apply(count_sentences)

print(df_medium[['message', 'n_sentences']].head())

# Analyse eksempel

In [ ]:
comparison = df_medium.groupby('label').agg({
    'n_tokens': 'mean',
    'n_verbs': 'mean',
    'n_nouns': 'mean',
    'n_entities': 'mean',
    'n_sentences': 'mean'
}).round(2)

print("\nGennemsnitlige værdier:")
print(comparison)

In [ ]:
def keep_only_content_words(doc):
    """Behold kun substantiver, verber, adjektiver og adverbier"""
    keep_tags = ['NOUN', 'VERB', 'ADJ', 'ADV']
    return [token.lemma_ for token in doc if token.pos_ in keep_tags]

df_medium['content_words'] = df_medium['doc'].apply(keep_only_content_words)


In [ ]:
print("\Original vs filtreret:")
for i in range(3):
    print(f"\nBesked {i+1}:")
    print(f"Original: {df_medium['message'].iloc[i][:80]}...")
    print(f"Content words: {' '.join(df_medium['content_words'].iloc[i][:15])}")


### Optimering af pipeline

In [ ]:
# Hvis vi kun skal bruge tokens og POS tags, kan vi deaktivere parser og NER
nlp_optimized = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
df_test = df.head(500).copy()
docs_optimized = list(nlp_optimized.pipe(df_test['message']))

### For store datasets (>10,000 rækker)

1. BRUG .pipe() I STEDET FOR .apply()
2. BATCH SIZE: list(nlp.pipe(texts, batch_size=100))
3. DEAKTIVER UNØDVENDIGE KOMPONENTER
5. GEM RESULTATER (Genberegn ikke hver gang)


In [ ]:
def process_large_dataframe(df, text_column, batch_size=100):
    """Optimeret funktion til at processere store DataFrames"""
    
    # Brug optimeret pipeline
    nlp_fast = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    
    # Process med pipe og batch_size
    docs = list(nlp_fast.pipe(df[text_column], batch_size=batch_size))
    
    # Udtræk features direkte
    results = {
        'tokens': [[token.text for token in doc] for doc in docs],
        'lemmas': [[token.lemma_ for token in doc] for doc in docs],
        'pos_tags': [[token.pos_ for token in doc] for doc in docs],
        'n_tokens': [len(doc) for doc in docs]
    }
    
    return results

In [ ]:
df_large = df.head(1000).copy()

In [ ]:
results = process_large_dataframe(df_large, 'message', batch_size=100)

In [ ]:
# Tilføj resultater til dataframe
for key, values in results.items():
    df_large[key] = values

In [ ]:
# Gem dataframe med alle features
# Fjern 'doc' kolonnen da den ikke kan gemmes direkte

#df_to_save = df_medium.drop('doc', axis=1)
#df_to_save.to_csv('spam_spacy_features.csv', index=False, encoding='utf-8')


# Bud på en meningsfuld analyse

> **Bruger spam-beskeder mere imperative verber (kommandoer) end normale beskeder?**

> Indeholder spam flere entiteter som navne/organisationer for at virke legitime?

> Er spam-beskeder kortere og mere "aggressive" i deres sprogbrug?

> **Hvilke konkrete ord og grammatiske mønstre karakteriserer spam?**

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['ner'])  # Aktivér ner igen senere

In [ ]:
# Process beskederne (brug subset for test, fjern [:1000] for fuld analyse)
df_analysis = df[:1000].copy()
docs = list(nlp.pipe(df_analysis['message'], batch_size=50))
df_analysis['doc'] = docs

In [ ]:
# Basis features
df_analysis['n_tokens'] = df_analysis['doc'].apply(lambda doc: len(doc))
df_analysis['n_sentences'] = df_analysis['doc'].apply(lambda doc: len(list(doc.sents)))
df_analysis['avg_sentence_length'] = df_analysis['n_tokens'] / df_analysis['n_sentences']


In [ ]:
# VERB features (vigtige for at identificere kommandoer)
def count_imperatives(doc):
    """Tæl imperative verber (kommandoer som 'Call', 'Text', 'Click')"""
    imperatives = 0
    for sent in doc.sents:
        tokens = list(sent)
        if len(tokens) > 0:
            # Imperativer starter ofte sætningen og er verber
            if tokens[0].pos_ == 'VERB' and tokens[0].tag_ == 'VB':
                imperatives += 1
    return imperatives

def count_verbs_by_type(doc):
    """Tæl forskellige typer af verber"""
    verb_types = {'VB': 0, 'VBG': 0, 'VBD': 0, 'VBN': 0, 'VBP': 0, 'VBZ': 0}
    for token in doc:
        if token.tag_ in verb_types:
            verb_types[token.tag_] += 1
    return verb_types

In [ ]:
df_analysis['n_imperatives'] = df_analysis['doc'].apply(count_imperatives)
df_analysis['n_verbs'] = df_analysis['doc'].apply(lambda doc: sum(1 for t in doc if t.pos_ == 'VERB'))
df_analysis['verb_ratio'] = df_analysis['n_verbs'] / df_analysis['n_tokens']

In [ ]:
# Urgency markers (ord der skaber tidspres)
urgency_words = ['now', 'urgent', 'immediately', 'today', 'limited', 'expires', 'hurry', 'asap', 'quick']

def count_urgency_words(doc):
    """Tæl ord der skaber følelse af tidspres"""
    text_lower = doc.text.lower()
    return sum(1 for word in urgency_words if word in text_lower)

In [ ]:
df_analysis['n_urgency'] = df_analysis['doc'].apply(count_urgency_words)

In [ ]:
# Incentive markers (belønninger og gratis tilbud)
incentive_words = ['free', 'win', 'prize', 'bonus', 'gift', 'cash', 'claim', 'reward']

def count_incentive_words(doc):
    """Tæl ord der lover belønninger"""
    text_lower = doc.text.lower()
    return sum(1 for word in incentive_words if word in text_lower)

In [ ]:
df_analysis['n_incentives'] = df_analysis['doc'].apply(count_incentive_words)

In [ ]:
# Special characters (tegn som !, ?, £, $)
def count_special_chars(text):
    """Tæl særlige tegn der bruges til at tiltrække opmærksomhed"""
    special = ['!', '?', '£', '$', '*', '#']
    return sum(text.count(char) for char in special)

In [ ]:
df_analysis['n_special_chars'] = df_analysis['message'].apply(count_special_chars)

In [ ]:
# Tal og numre (ofte telefonnumre, koder)
def count_numbers(doc):
    """Tæl numeriske tokens"""
    return sum(1 for token in doc if token.like_num or token.is_digit)

In [ ]:
df_analysis['n_numbers'] = df_analysis['doc'].apply(count_numbers)

### Sammenligning af sproglige mønstre

In [ ]:
comparison = df_analysis.groupby('label').agg({
    'n_tokens': 'mean',
    'n_sentences': 'mean',
    'avg_sentence_length': 'mean',
    'n_imperatives': 'mean',
    'verb_ratio': 'mean',
    'n_urgency': 'mean',
    'n_incentives': 'mean',
    'n_special_chars': 'mean',
    'n_numbers': 'mean'
}).round(3)

print("\nGennemsnitlige værdier (spam vs ham):")
print(comparison)

In [ ]:
# Beregn forskelle
spam_data = df_analysis[df_analysis['label'] == 'spam']
ham_data = df_analysis[df_analysis['label'] == 'ham']

In [ ]:
print(f"\nANALYSE 1. IMPERATIVER (kommandoer):")
print(f"   Spam: {spam_data['n_imperatives'].mean():.2f} per besked")
print(f"   Ham: {ham_data['n_imperatives'].mean():.2f} per besked")
print(f"   Spam bruger {(spam_data['n_imperatives'].mean() / ham_data['n_imperatives'].mean()):.1f}x flere kommandoer")

print(f"\nANALYSE 2. URGENCY (tidspres):")
print(f"   Spam: {spam_data['n_urgency'].mean():.2f} urgency-ord per besked")
print(f"   Ham: {ham_data['n_urgency'].mean():.2f} urgency-ord per besked")
spam_urgency_pct = (spam_data['n_urgency'] > 0).sum() / len(spam_data) * 100
ham_urgency_pct = (ham_data['n_urgency'] > 0).sum() / len(ham_data) * 100
print(f"   {spam_urgency_pct:.1f}% af spam bruger urgency-sprog vs {ham_urgency_pct:.1f}% af ham")

print(f"\nANALYSE 3. INCENTIVER (belønninger):")
print(f"   Spam: {spam_data['n_incentives'].mean():.2f} incentive-ord per besked")
print(f"   Ham: {ham_data['n_incentives'].mean():.2f} incentive-ord per besked")
spam_incentive_pct = (spam_data['n_incentives'] > 0).sum() / len(spam_data) * 100
print(f"   {spam_incentive_pct:.1f}% af spam lover belønninger")

print(f"\nANALYSE 4. SÆRLIGE TEGN (!, £, $):")
print(f"   Spam: {spam_data['n_special_chars'].mean():.2f} per besked")
print(f"   Ham: {ham_data['n_special_chars'].mean():.2f} per besked")

In [ ]:
def get_content_words(doc):
    """Udtræk indholdsord (substantiver, verber, adjektiver)"""
    return [token.lemma_.lower() for token in doc 
            if token.pos_ in ['NOUN', 'VERB', 'ADJ'] 
            and not token.is_stop 
            and token.is_alpha]

spam_words = []
ham_words = []

for idx, row in df_analysis.iterrows():
    words = get_content_words(row['doc'])
    if row['label'] == 'spam':
        spam_words.extend(words)
    else:
        ham_words.extend(words)

spam_top = Counter(spam_words).most_common(15)
ham_top = Counter(ham_words).most_common(15)

print("\nTop 15 ord i SPAM:")
for word, count in spam_top:
    print(f"  {word:15} ({count})")

print("\nTop 15 ord i HAM:")
for word, count in ham_top:
    print(f"  {word:15} ({count})")

## Visualisering af resultater

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Sproglige Manipulationsteknikker i Spam-beskeder', 
             fontsize=16, fontweight='bold')

# Imperatives comparison
axes[0, 0].bar(['Ham', 'Spam'], 
               [ham_data['n_imperatives'].mean(), spam_data['n_imperatives'].mean()],
               color=['#2ecc71', '#e74c3c'], edgecolor='black', linewidth=1.5)
axes[0, 0].set_ylabel('Gennemsnit per besked')
axes[0, 0].set_title('Imperative Verber (Kommandoer)')
axes[0, 0].grid(axis='y', alpha=0.3)

# Urgency words
axes[0, 1].bar(['Ham', 'Spam'],
               [ham_data['n_urgency'].mean(), spam_data['n_urgency'].mean()],
               color=['#2ecc71', '#e74c3c'], edgecolor='black', linewidth=1.5)
axes[0, 1].set_ylabel('Gennemsnit per besked')
axes[0, 1].set_title('Urgency-ord (Tidspres)')
axes[0, 1].grid(axis='y', alpha=0.3)

# Incentive words
axes[0, 2].bar(['Ham', 'Spam'],
               [ham_data['n_incentives'].mean(), spam_data['n_incentives'].mean()],
               color=['#2ecc71', '#e74c3c'], edgecolor='black', linewidth=1.5)
axes[0, 2].set_ylabel('Gennemsnit per besked')
axes[0, 2].set_title('Incentive-ord (Belønninger)')
axes[0, 2].grid(axis='y', alpha=0.3)

# Message length distribution
axes[1, 0].hist(ham_data['n_tokens'], bins=30, alpha=0.6, label='Ham', color='#2ecc71')
axes[1, 0].hist(spam_data['n_tokens'], bins=30, alpha=0.6, label='Spam', color='#e74c3c')
axes[1, 0].set_xlabel('Antal ord')
axes[1, 0].set_ylabel('Antal beskeder')
axes[1, 0].set_title('Beskedlængde')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# Special characters
axes[1, 1].bar(['Ham', 'Spam'],
               [ham_data['n_special_chars'].mean(), spam_data['n_special_chars'].mean()],
               color=['#2ecc71', '#e74c3c'], edgecolor='black', linewidth=1.5)
axes[1, 1].set_ylabel('Gennemsnit per besked')
axes[1, 1].set_title('Særlige Tegn (!, £, $)')
axes[1, 1].grid(axis='y', alpha=0.3)

# Composite spam score
axes[1, 2].scatter(spam_data['n_urgency'] + spam_data['n_incentives'], 
                   spam_data['n_imperatives'],
                   alpha=0.5, color='#e74c3c', label='Spam', s=30)
axes[1, 2].scatter(ham_data['n_urgency'] + ham_data['n_incentives'],
                   ham_data['n_imperatives'],
                   alpha=0.5, color='#2ecc71', label='Ham', s=30)
axes[1, 2].set_xlabel('Urgency + Incentive ord')
axes[1, 2].set_ylabel('Imperative verber')
axes[1, 2].set_title('Manipulationsmønstre')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()